In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

In [20]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-05 19:27:36,425 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 19:27:36,440 INFO: Initializing external client
2025-03-05 19:27:36,441 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-03-05 19:27:36,918 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214711
Fetching data from 2025-02-05 00:27:36.425461+00:00 to 2025-03-05 23:27:36.425461+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.70s) 


In [21]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-05 19:00:00+00:00,2,0
1,2025-02-05 20:00:00+00:00,2,0
2,2025-02-05 21:00:00+00:00,2,0
3,2025-02-05 22:00:00+00:00,2,0
4,2025-02-05 23:00:00+00:00,2,0
...,...,...,...
170599,2025-03-05 19:00:00+00:00,263,161
170600,2025-03-05 20:00:00+00:00,263,123
170601,2025-03-05 21:00:00+00:00,263,109
170602,2025-03-05 22:00:00+00:00,263,82


In [22]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 170604 entries, 0 to 170855
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         170604 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  170604 non-null  int32                  
 2   rides               170604 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 3.9 MB


In [23]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [24]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 170604 entries, 0 to 170855
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         170604 non-null  datetime64[us]
 1   pickup_location_id  170604 non-null  int32         
 2   rides               170604 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 3.9 MB


In [25]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [26]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,3,0,0,0,0,0,2,0,3,2,...,0,2,2,0,1,0,0,2,216,2025-02-11 14:00:00
1,2,0,0,0,0,0,0,0,0,0,...,0,0,2,2,0,1,0,0,217,2025-03-05 02:00:00
2,0,50,0,34,10,0,32,35,7,6,...,2,9,6,31,46,1,12,8,261,2025-02-10 09:00:00
3,3,5,3,0,5,0,2,2,1,0,...,6,3,1,5,0,0,2,0,65,2025-02-25 21:00:00
4,0,0,0,0,0,0,1,1,1,0,...,1,0,2,0,0,0,0,1,56,2025-02-18 14:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,30,2025-02-28 14:00:00
248,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,73,2025-02-22 19:00:00
249,3,0,1,0,0,0,0,0,3,2,...,0,0,4,5,0,1,1,0,35,2025-02-07 09:00:00
250,4,5,2,8,10,2,0,8,1,4,...,0,4,9,7,0,11,0,5,4,2025-02-19 03:00:00


In [27]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-05 19:29:42,927 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 19:29:42,933 INFO: Initializing external client
2025-03-05 19:29:42,933 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-03-05 19:29:43,550 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214711
Fetching data from 2025-02-05 00:29:42.927057+00:00 to 2025-03-05 23:29:42.927057+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.28s) 


In [28]:
current_date

Timestamp('2025-03-06 00:29:42.927057+0000', tz='Etc/UTC')

In [29]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-05 19:00:00
1,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,3,2025-03-05 19:00:00
2,6,5,4,4,4,0,3,1,0,1,...,3,5,4,3,5,7,18,20,4,2025-03-05 19:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,6,2025-03-05 19:00:00
4,2,1,1,0,1,2,1,1,0,0,...,2,1,2,2,8,4,1,8,7,2025-03-05 19:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,0,0,0,0,0,0,0,0,0,1,...,1,0,0,1,0,0,0,0,259,2025-03-05 19:00:00
248,1,0,1,2,0,2,1,0,2,0,...,1,2,0,4,2,0,0,2,260,2025-03-05 19:00:00
249,43,37,20,31,10,4,1,0,0,2,...,28,35,43,42,43,59,40,50,261,2025-03-05 19:00:00
250,78,35,38,13,8,2,2,1,0,6,...,109,105,97,100,134,169,178,163,262,2025-03-05 19:00:00


In [30]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-05 19:30:06,487 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 19:30:06,492 INFO: Initializing external client
2025-03-05 19:30:06,493 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 19:30:06,988 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214711


In [31]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [32]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,8.0
3,6,0.0
4,7,3.0
...,...,...
247,259,0.0
248,260,1.0
249,261,40.0
250,262,83.0


In [33]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([161, 237, 162, 230, 236, 142, 170, 234, 163, 138], dtype=int32)